# RNA 3D Feature Extraction - Refactored Workflow Demonstration

This notebook demonstrates the refactored RNA 3D Feature Extractor architecture, showing how the modular components work together to extract, validate, and analyze RNA features.

## Refactored Architecture

The refactored codebase is organized into these primary modules:

- **DataManager**: Handles data loading, saving, and I/O operations
- **FeatureExtractor**: Manages thermodynamic and MI feature extraction
- **BatchProcessor**: Coordinates processing multiple RNA targets
- **MemoryMonitor**: Tracks memory usage during processing
- **ResultValidator**: Validates the quality and compatibility of features

These components work together through the integrated **RNAFeatureExtractionWorkflow** class to provide a complete feature extraction pipeline.

In [ ]:
# Import standard libraries
import os
import sys
import time
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Add project root to path to allow importing modules
if os.path.basename(os.getcwd()) == "notebooks":
    # If we're in the notebooks directory, go up one level
    project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
else:
    # Otherwise, assume we're in the project root
    project_root = os.getcwd()
    
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our modules
from src.data.data_manager import DataManager
from src.features.feature_extractor import FeatureExtractor
from src.processing.batch_processor import BatchProcessor
from src.analysis.memory_monitor import MemoryMonitor
from src.validation.result_validator import ResultValidator
from src.workflow import RNAFeatureExtractionWorkflow

print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

## 1. Using Individual Components

First, let's see how each component works individually.

### 1.1 DataManager

The DataManager handles data loading, saving, and I/O operations.

In [ ]:
# Create a DataManager instance
data_manager = DataManager(base_dir="data")

# List available target IDs
csv_path = "data/targets.csv"
if os.path.exists(csv_path):
    targets_df = data_manager.load_rna_data(csv_path)
    target_ids = data_manager.get_unique_target_ids(targets_df)
    print(f"Found {len(target_ids)} target IDs in {csv_path}")
    print(f"First 5 target IDs: {target_ids[:5]}")
else:
    # Define a sample target ID for demonstration
    target_ids = ["R1107", "R1108", "R1117v2"]
    print(f"Using sample target IDs: {target_ids}")
    
# Set up a test target ID for demonstration
if target_ids:
    test_target_id = target_ids[0]
    print(f"Using test target ID: {test_target_id}")
else:
    test_target_id = "R1107"
    print(f"Using default test target ID: {test_target_id}")

# Try to load the sequence for the test target
try:
    sequence = data_manager.get_sequence_for_target(test_target_id)
    print(f"Loaded sequence for {test_target_id}: {sequence[:50]}... (length: {len(sequence)})")
except Exception as e:
    print(f"Could not load sequence for {test_target_id}: {e}")
    # Create a dummy sequence for demonstration
    sequence = "GAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUCGAUC"
    print(f"Using dummy sequence: {sequence[:50]}... (length: {len(sequence)})")

# Try to load MSA data for the test target
try:
    msa_sequences = data_manager.load_msa_data(test_target_id)
    print(f"Loaded MSA data for {test_target_id}: {len(msa_sequences)} sequences")
except Exception as e:
    print(f"Could not load MSA data for {test_target_id}: {e}")
    # Create dummy MSA sequences for demonstration
    msa_sequences = [sequence] * 3  # Single-sequence MSA
    print(f"Using dummy MSA data: {len(msa_sequences)} sequences")

### 1.2 MemoryMonitor

The MemoryMonitor tracks memory usage during processing.

In [ ]:
# Create a MemoryMonitor instance
memory_monitor = MemoryMonitor()

# Log current memory usage
memory_monitor.log_memory_usage("Initial memory usage")

# Demonstrate memory tracking with context manager
with memory_monitor.track("Allocating memory"):
    # Allocate some memory to see the effect
    data = [0] * 10**7
    time.sleep(1)

# Get current memory usage
current_memory = memory_monitor.get_current_memory_usage()
print(f"Current memory usage: {current_memory:.2f} GB")

# Plot memory usage
memory_monitor.plot_memory_usage()

### 1.3 FeatureExtractor

The FeatureExtractor handles extracting thermodynamic and mutual information features.

In [ ]:
# Create a FeatureExtractor instance
feature_extractor = FeatureExtractor()

# Extract thermodynamic features for a short sequence to demonstrate
demo_sequence = "GGGAAAACCC"  # Short sequence for demonstration
print(f"Extracting thermodynamic features for sequence: {demo_sequence}")

with memory_monitor.track("Extracting thermodynamic features"):
    thermo_features = feature_extractor.extract_thermodynamic_features(demo_sequence)

# Display the extracted thermodynamic features
print("\nExtracted thermodynamic features:")
for key, value in thermo_features.items():
    if isinstance(value, np.ndarray) and value.ndim > 0 and value.size > 10:
        print(f"  {key}: ndarray with shape {value.shape}")
    else:
        print(f"  {key}: {value}")

# Extract MI features for a simple MSA to demonstrate
demo_msa = [
    "GGGAAAACCC",
    "GGGAAAACCC",
    "GGGAAAACCC"
]
print(f"\nExtracting MI features for MSA with {len(demo_msa)} sequences")

with memory_monitor.track("Extracting MI features"):
    mi_features = feature_extractor.extract_mi_features(demo_msa)

# Display the extracted MI features
print("\nExtracted MI features:")
for key, value in mi_features.items():
    if isinstance(value, np.ndarray) and value.ndim > 0 and value.size > 10:
        print(f"  {key}: ndarray with shape {value.shape}")
    else:
        print(f"  {key}: {value}")

# Extract all features at once
print(f"\nExtracting all features for test target: {test_target_id}")

with memory_monitor.track("Extracting all features"):
    all_features = feature_extractor.extract_features(
        target_id=test_target_id,
        sequence=sequence,
        msa_sequences=msa_sequences,
        extract_thermo=True,
        extract_mi=True
    )

# Display the feature types extracted
print("\nExtracted feature types:")
for feature_type, features in all_features.items():
    print(f"  {feature_type}: {len(features)} features")

### 1.4 ResultValidator

The ResultValidator validates the quality and compatibility of features.

In [ ]:
# Create a ResultValidator instance
result_validator = ResultValidator(data_manager=data_manager)

# Validate the thermodynamic features we just extracted
print("Validating thermodynamic features:")
thermo_validation = result_validator.validate_thermodynamic_features(thermo_features)
print(f"  Valid: {thermo_validation['is_valid']}")
print(f"  Issues: {thermo_validation['issues']}")
print(f"  Warnings: {thermo_validation['warnings']}")
print(f"  Stats: {thermo_validation['stats']}")

# Validate MI features
print("\nValidating MI features:")
mi_validation = result_validator.validate_mi_features(mi_features)
print(f"  Valid: {mi_validation['is_valid']}")
print(f"  Issues: {mi_validation['issues']}")
print(f"  Warnings: {mi_validation['warnings']}")
print(f"  Stats: {mi_validation['stats']}")

# Validate feature compatibility
print("\nValidating feature compatibility:")
compatibility_validation = result_validator.validate_feature_compatibility({
    "thermo_features": thermo_features,
    "mi_features": mi_features
})
print(f"  Valid: {compatibility_validation['is_valid']}")
print(f"  Issues: {compatibility_validation['issues']}")
print(f"  Warnings: {compatibility_validation['warnings']}")

### 1.5 BatchProcessor

The BatchProcessor coordinates processing multiple RNA targets.

In [ ]:
# Create a BatchProcessor instance
batch_processor = BatchProcessor(
    data_manager=data_manager,
    feature_extractor=feature_extractor,
    memory_monitor=memory_monitor,
    output_dir="data/processed",
    log_dir="data/processed/logs",
    max_memory_usage_gb=16.0,
    batch_size=5
)

# Process a small batch of targets (if available, otherwise use dummy targets)
if len(target_ids) >= 3:
    batch_target_ids = target_ids[:3]
else:
    batch_target_ids = ["dummy_target_1", "dummy_target_2", "dummy_target_3"]

print(f"Processing batch of {len(batch_target_ids)} targets: {batch_target_ids}")

# For demonstration purposes, allow the code to continue even if the batch processing fails
try:
    with memory_monitor.track("Batch processing"):
        batch_results = batch_processor.process_targets(
            target_ids=batch_target_ids,
            extract_thermo=True,
            extract_mi=True,
            save_intermediates=True,
            batch_name="demo_batch"
        )
    
    # Display batch processing results
    print("\nBatch processing results:")
    print(f"  Total targets: {batch_results['total_targets']}")
    print(f"  Successful targets: {batch_results['successful_targets']}")
    print(f"  Skipped targets: {batch_results['skipped_targets']}")
    
    # Validate batch results
    batch_validation = result_validator.validate_batch_results(batch_results)
    print("\nBatch validation results:")
    print(f"  Valid targets: {batch_validation['valid_targets']}")
    print(f"  Invalid targets: {batch_validation['invalid_targets']}")
    print(f"  Targets with warnings: {batch_validation['targets_with_warnings']}")
except Exception as e:
    print(f"\nBatch processing failed: {e}")
    print("Continuing with demonstration using the integrated workflow...")

## 2. Integrated Workflow

Now, let's use the integrated RNAFeatureExtractionWorkflow class to run the complete pipeline.

In [ ]:
# Create a workflow instance
workflow = RNAFeatureExtractionWorkflow(
    data_dir="data",
    output_dir="data/processed",
    log_dir="data/processed/logs",
    memory_plot_dir="data/processed/memory_plots",
    validation_report_dir="data/processed/validation_reports",
    max_memory_gb=16.0,
    batch_size=5
)

### 2.1 Extracting Features for a Single Target

In [ ]:
# Extract features for a single target
print(f"Extracting features for single target: {test_target_id}")

try:
    single_target_results = workflow.extract_single_target(
        target_id=test_target_id,
        extract_thermo=True,
        extract_mi=True,
        validate_results=True,
        save_memory_plot=True
    )
    
    # Display results
    print("\nSingle target extraction results:")
    print(f"  Target ID: {single_target_results['target_id']}")
    print(f"  Features extracted: {single_target_results['features_extracted']}")
    print(f"  Execution time: {single_target_results['execution_time']:.2f} seconds")
    print(f"  Peak memory usage: {single_target_results['peak_memory_gb']:.2f} GB")
    
    if 'validation' in single_target_results:
        print("  Validation:")
        print(f"    Valid: {single_target_results['validation']['is_valid']}")
        print(f"    Issues: {single_target_results['validation']['issues_count']}")
        print(f"    Warnings: {single_target_results['validation']['warnings_count']}")
except Exception as e:
    print(f"\nSingle target extraction failed: {e}")
    print("This could be due to missing data files. Continuing with demonstration...")

### 2.2 Create Targets File for Batch Processing

In [ ]:
# Create a targets file for demonstration
targets_file = "data/demo_targets.txt"
os.makedirs(os.path.dirname(targets_file), exist_ok=True)

# Write target IDs to the file (use real targets if available, otherwise use dummy targets)
if len(target_ids) >= 3:
    demo_targets = target_ids[:3]
else:
    demo_targets = ["dummy_target_1", "dummy_target_2", "dummy_target_3"]

with open(targets_file, 'w') as f:
    for target_id in demo_targets:
        f.write(f"{target_id}\n")

print(f"Created targets file: {targets_file} with {len(demo_targets)} targets")

### 2.3 Running the Complete Workflow

In [ ]:
# Run the complete workflow
print(f"Running complete workflow for targets in {targets_file}")

try:
    workflow_results = workflow.run_extraction(
        targets_file=targets_file,
        extract_thermo=True,
        extract_mi=True,
        batch_name="demo_workflow",
        validate_results=True,
        save_memory_plots=True
    )
    
    # Display workflow results
    print("\nWorkflow results:")
    print(f"  Batch name: {workflow_results['batch_name']}")
    print(f"  Total targets: {workflow_results['total_targets']}")
    print(f"  Successful targets: {workflow_results['successful_targets']}")
    print(f"  Skipped targets: {workflow_results['skipped_targets']}")
    print(f"  Execution time: {workflow_results['execution_time']:.2f} seconds")
    print(f"  Peak memory usage: {workflow_results['peak_memory_gb']:.2f} GB")
    
    if 'validation' in workflow_results:
        print("  Validation:")
        print(f"    Valid targets: {workflow_results['validation']['valid_targets']}")
        print(f"    Invalid targets: {workflow_results['validation']['invalid_targets']}")
        print(f"    Targets with warnings: {workflow_results['validation']['targets_with_warnings']}")
except Exception as e:
    print(f"\nWorkflow execution failed: {e}")
    print("This could be due to missing data files or other issues.")

## 3. Analyzing Results

Let's load and analyze the results of our feature extraction.

In [ ]:
# Check if we have output files
thermo_features_dir = "data/processed/thermo_features"
mi_features_dir = "data/processed/mi_features"

# List available feature files
thermo_files = []
mi_files = []

if os.path.exists(thermo_features_dir):
    thermo_files = [f for f in os.listdir(thermo_features_dir) if f.endswith(".npz")]
    print(f"Found {len(thermo_files)} thermodynamic feature files")
else:
    print(f"Thermodynamic features directory not found: {thermo_features_dir}")
    
if os.path.exists(mi_features_dir):
    mi_files = [f for f in os.listdir(mi_features_dir) if f.endswith(".npz")]
    print(f"Found {len(mi_files)} MI feature files")
else:
    print(f"MI features directory not found: {mi_features_dir}")

# If we have feature files, load and display one
if thermo_files:
    sample_thermo_file = os.path.join(thermo_features_dir, thermo_files[0])
    print(f"\nLoading sample thermodynamic features file: {sample_thermo_file}")
    
    thermo_features = np.load(sample_thermo_file, allow_pickle=True)
    print("Feature keys:")
    for key in thermo_features.keys():
        feature = thermo_features[key]
        if isinstance(feature, np.ndarray) and feature.ndim > 0 and feature.size > 10:
            print(f"  {key}: ndarray with shape {feature.shape}")
        else:
            print(f"  {key}: {feature}")
            
if mi_files:
    sample_mi_file = os.path.join(mi_features_dir, mi_files[0])
    print(f"\nLoading sample MI features file: {sample_mi_file}")
    
    mi_features = np.load(sample_mi_file, allow_pickle=True)
    print("Feature keys:")
    for key in mi_features.keys():
        feature = mi_features[key]
        if isinstance(feature, np.ndarray) and feature.ndim > 0 and feature.size > 10:
            print(f"  {key}: ndarray with shape {feature.shape}")
        else:
            print(f"  {key}: {feature}")

## 4. Visualizing Features

Let's visualize some of the extracted features.

In [ ]:
# Visualize pairing probabilities matrix if available
if 'thermo_features' in locals() and 'struct.pairing_probs' in thermo_features:
    pairing_probs = thermo_features['struct.pairing_probs']
    
    plt.figure(figsize=(10, 8))
    plt.imshow(pairing_probs, cmap='viridis')
    plt.colorbar(label='Pairing Probability')
    plt.title('RNA Base Pairing Probabilities')
    plt.xlabel('Nucleotide Position')
    plt.ylabel('Nucleotide Position')
    plt.tight_layout()
    plt.show()
elif 'thermo_features' in locals() and 'pairing_probs' in thermo_features:
    pairing_probs = thermo_features['pairing_probs']
    
    plt.figure(figsize=(10, 8))
    plt.imshow(pairing_probs, cmap='viridis')
    plt.colorbar(label='Pairing Probability')
    plt.title('RNA Base Pairing Probabilities')
    plt.xlabel('Nucleotide Position')
    plt.ylabel('Nucleotide Position')
    plt.tight_layout()
    plt.show()
else:
    print("Pairing probabilities matrix not available for visualization")
    
# Visualize MI coupling matrix if available
if 'mi_features' in locals() and 'mi.coupling_matrix' in mi_features:
    coupling_matrix = mi_features['mi.coupling_matrix']
    
    plt.figure(figsize=(10, 8))
    plt.imshow(coupling_matrix, cmap='plasma')
    plt.colorbar(label='Mutual Information')
    plt.title('RNA Mutual Information Coupling Matrix')
    plt.xlabel('Nucleotide Position')
    plt.ylabel('Nucleotide Position')
    plt.tight_layout()
    plt.show()
elif 'mi_features' in locals() and 'coupling_matrix' in mi_features:
    coupling_matrix = mi_features['coupling_matrix']
    
    plt.figure(figsize=(10, 8))
    plt.imshow(coupling_matrix, cmap='plasma')
    plt.colorbar(label='Mutual Information')
    plt.title('RNA Mutual Information Coupling Matrix')
    plt.xlabel('Nucleotide Position')
    plt.ylabel('Nucleotide Position')
    plt.tight_layout()
    plt.show()
else:
    print("MI coupling matrix not available for visualization")

## 5. Review of Refactored Architecture

The refactored RNA 3D Feature Extractor architecture provides several key advantages:

1. **Modularity**: Each component has a well-defined responsibility, making the code easier to understand, maintain, and extend.

2. **Memory Efficiency**: The architecture includes built-in memory monitoring and optimization, with the ability to process targets in batches to prevent out-of-memory errors.

3. **Validation**: The ResultValidator component ensures that extracted features meet quality standards and are compatible with downstream models.

4. **Fault Tolerance**: The system can recover from errors in individual targets and continue processing other targets.

5. **Flexibility**: The architecture supports different feature types, data sources, and processing configurations.

6. **Maintainability**: Clear separation of concerns makes it easier to update or replace individual components without affecting the rest of the system.

7. **Performance Monitoring**: Built-in memory and performance tracking helps identify bottlenecks and optimize resource usage.

This refactored architecture provides a solid foundation for future enhancements and integrations.